In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# -------------------------- LIBRARIES NECESSARY IN THIS PROJECT  -------------------------- #

import pandas as pd
import numpy as np
import seaborn as sns
from datetime import *
import zipfile
import pylab as plt

import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from sklearn.tree import DecisionTreeClassifier

# -------------------------- FUNCTIONS -------------------------- #

def getDataSet(data_path):
    return pd.read_csv(data_path)

## Upload dataSets

### Members dataset cleasing

In [ ]:
# -------------------------- PATH OF THE DATASETS USED IN THE PROJECT  -------------------------- #
zf = zipfile.ZipFile('/content/drive/My Drive/dataSets/main.zip')

main_dataset = pd.read_csv(zf.open('main.csv'))

## Data manipulation and cleansing

In [ ]:
# Check the dataSet fields
print(f"Missing values in main_dataset each variable: \n{main_dataset.isnull().sum()}")
print(f"Duplicated rows: \n{main_dataset.duplicated().sum()}")
print(main_dataset.shape)

Missing values in main_dataset each variable: 
Unnamed: 0                    0
member_key                    0
updated_at                    0
first_name                    0
last_name                     0
disabilities              24567
SA1                       29199
price_zone_code               0
plan_key                      0
plan_status                   0
plan_start_date               0
plan_end_date                 0
budget_level3_name        32110
budget_level3_key         32110
budget_level2_key             0
budget_level2_name            0
budget_level1_key             0
budget_level1_name            0
invoice_state                 0
claim_state                   0
allocated_amount              0
requested_amount              0
funded_amount                 0
allocated_amount_total        0
requested_amount_total        0
funded_amount_total           0
dtype: int64
Duplicated rows: 
0
(32950, 26)


In [ ]:
main_dataset.dropna(axis=0,inplace=True)
main_dataset

,Unnamed: 0,member_key,updated_at,first_name,last_name,disabilities,SA1,price_zone_code,plan_key,plan_status,plan_start_date,plan_end_date,budget_level3_name,budget_level3_key,budget_level2_key,budget_level2_name,budget_level1_key,budget_level1_name,invoice_state,claim_state,allocated_amount,requested_amount,funded_amount,allocated_amount_total,requested_amount_total,funded_amount_total
447,447,04059890-e588-11ea-b453-3de7750d4ba6,2021-04,Gretel,Moreno,Cerebral palsy,3124802.0,ACT_NSW_QLD_VIC,0a67f700-e588-11ea-be19-95ba49e7f45d,PLAN_DELIVERY_ACTIVE,2020-08,2021-12,Coordination Of Supports,2d005c1d-754f-11ea-ade9-022376bbb7c0,7d0af54b-f597-11e9-bfb4-022d4762bb3c,Support coordination,5874551b-f597-11e9-bfb4-022d4762bb3c,Capacity Building,ALL_PAID,PAID,100.14,2428.4050,2428.41,554391.70,36088.7577,35031.29
1027,1027,0558f390-e588-11ea-b453-3de7750d4ba6,2021-04,Hali,Brislane,"Stroke, Kleinsfeld syndrome, occasional",7102006.0,ACT_NSW_QLD_VIC,09cf3970-e588-11ea-be19-95ba49e7f45d,COMPLETED,2020-07,2021-02,Assistive Products For Personal Care And Safety,ec637271-bb29-11ea-ade9-022376bbb7c0,7d0af628-f597-11e9-bfb4-022d4762bb3c,Assistive technology,587454f6-f597-11e9-bfb4-022d4762bb3c,Capital,ALL_PAID,PAID,7794.00,3897.0000,3897.00,948632.70,46177.8688,46177.87
1033,1033,0563f010-e588-11ea-b453-3de7750d4ba6,2021-04,Jesselyn,Munkton,"ADD, ADHD",0.0,ACT_NSW_QLD_VIC,98ec6b2e-dedf-4db8-a8d2-9ee8c44a4a5b,PLAN_DELIVERY_ACTIVE,2021-03,2022-09,Coordination Of Supports,2d005c1d-754f-11ea-ade9-022376bbb7c0,7d0af54b-f597-11e9-bfb4-022d4762bb3c,Support coordination,5874551b-f597-11e9-bfb4-022d4762bb3c,Capacity Building,ALL_APPROVED,SCHEME_AUTHORIZED,100.14,200.2800,0.00,65891.34,3546.1700,3345.89
1163,1163,05b28540-e588-11ea-b453-3de7750d4ba6,2021-04,Emilee,Styant,Alzheimer's dementia,0.0,NT_SA_TAS_WA,37ba070c-518e-4e23-b586-595c33092a22,PLAN_DELIVERY_ACTIVE,2020-08,2021-08,Assistance With Daily Life Tasks Provided In R...,2d001fef-754f-11ea-ade9-022376bbb7c0,7d0af71b-f597-11e9-bfb4-022d4762bb3c,Assistance with daily living,58745466-f597-11e9-bfb4-022d4762bb3c,Core,ALL_PAID,PAID,1.00,14315.0000,14315.00,99730.20,52108.1750,52639.18
1767,1767,13b3bd00-0485-11eb-aced-072c098d2335,2021-04,Matthiew,Buckett,Cerebral palsy,0.0,ACT_NSW_QLD_VIC,4af773f3-65ab-4015-b8ab-0b2b70b8d8cc,PLAN_DELIVERY_ACTIVE,2021-02,2022-02,Coordination Of Supports,2d005c1d-754f-11ea-ade9-022376bbb7c0,7d0af54b-f597-11e9-bfb4-022d4762bb3c,Support coordination,5874551b-f597-11e9-bfb4-022d4762bb3c,Capacity Building,ALL_APPROVED,PENDING_DISBURSEMENT,100.14,116.1624,0.00,469336.82,32211.6876,27648.31
1768,1768,13b3bd00-0485-11eb-aced-072c098d2335,2021-04,Matthiew,Buckett,Cerebral palsy,0.0,ACT_NSW_QLD_VIC,4af773f3-65ab-4015-b8ab-0b2b70b8d8cc,PLAN_DELIVERY_ACTIVE,2021-02,2022-02,Coordination Of Supports,2d005c1d-754f-11ea-ade9-022376bbb7c0,7d0af54b-f597-11e9-bfb4-022d4762bb3c,Support coordination,5874551b-f597-11e9-bfb4-022d4762bb3c,Capacity Building,ALL_PAID,PAID,100.14,25.0350,25.04,469336.82,32211.6876,27648.31
2631,2631,25703530-4f0a-11eb-aab6-0727412ff01c,2021-04,Shirline,Purvis,Brain Injury,3124802.0,ACT_NSW_QLD_VIC,99accb3d-0d8c-437b-9625-416d0e3d008e,PLAN_DELIVERY_ACTIVE,2020-12,2021-12,Consultation About Home Modification Designs W...,85e9dec0-754f-11ea-ade9-022376bbb7c0,7d0af64d-f597-11e9-bfb4-022d4762bb3c,Home,587454f6-f597-11e9-bfb4-022d4762bb3c,Capital,ALL_PAID,PAID,500.00,306.3000,306.30,68850.62,5995.9875,5996.08
5248,5248,5a853620-09f1-11eb-b7e6-5f90c3a4b38d,2021-04,Levy,Salle,"Depression, Acquired Brain Injury,",3131308.0,ACT_NSW_QLD_VIC,8ca47034-ef3c-45fa-8549-1aae25611155,PLAN_DELIVERY_ACTIVE,2020-10,2021-10,Coordination Of Supports,2d005c1d-754f-11ea-ade9-022376bbb7c0,7d0af54b-f597-11e9-bfb4-022d4762bb3c,Support coordination,5874551b-f597-11e9-bfb4-022d4762bb3c,Capacity Building,ALL_PAID,PAID,100.14,6208.4448,6208.44,4295076.49,172764.7611,174281.69
8843,8843,75cc8ab0-70b6-11eb-ad13-9558bc5f84e9,2021-04,Burch,Clelle,Intellectual disability,0.0,ACT_NSW_QLD_VIC,fc9c

In [ ]:
main_dataset = main_dataset.replace(np.nan, 'NA')

main_dataset = main_dataset[(main_dataset["plan_status"] == 'PLAN_DELIVERY_ACTIVE')]
main_dataset = main_dataset[(main_dataset["claim_state"] == 'PAID')]
main_dataset

,Unnamed: 0,member_key,updated_at,first_name,last_name,disabilities,SA1,price_zone_code,plan_key,plan_status,plan_start_date,plan_end_date,budget_level3_name,budget_level3_key,budget_level2_key,budget_level2_name,budget_level1_key,budget_level1_name,invoice_state,claim_state,allocated_amount,requested_amount,funded_amount,allocated_amount_total,requested_amount_total,funded_amount_total
447,447,04059890-e588-11ea-b453-3de7750d4ba6,2021-04,Gretel,Moreno,Cerebral palsy,3124802.0,ACT_NSW_QLD_VIC,0a67f700-e588-11ea-be19-95ba49e7f45d,PLAN_DELIVERY_ACTIVE,2020-08,2021-12,Coordination Of Supports,2d005c1d-754f-11ea-ade9-022376bbb7c0,7d0af54b-f597-11e9-bfb4-022d4762bb3c,Support coordination,5874551b-f597-11e9-bfb4-022d4762bb3c,Capacity Building,ALL_PAID,PAID,100.14,2428.4050,2428.41,554391.70,36088.7577,35031.29
1163,1163,05b28540-e588-11ea-b453-3de7750d4ba6,2021-04,Emilee,Styant,Alzheimer's dementia,0.0,NT_SA_TAS_WA,37ba070c-518e-4e23-b586-595c33092a22,PLAN_DELIVERY_ACTIVE,2020-08,2021-08,Assistance With Daily Life Tasks Provided In R...,2d001fef-754f-11ea-ade9-022376bbb7c0,7d0af71b-f597-11e9-bfb4-022d4762bb3c,Assistance with daily living,58745466-f597-11e9-bfb4-022d4762bb3c,Core,ALL_PAID,PAID,1.00,14315.0000,14315.00,99730.20,52108.1750,52639.18
1768,1768,13b3bd00-0485-11eb-aced-072c098d2335,2021-04,Matthiew,Buckett,Cerebral palsy,0.0,ACT_NSW_QLD_VIC,4af773f3-65ab-4015-b8ab-0b2b70b8d8cc,PLAN_DELIVERY_ACTIVE,2021-02,2022-02,Coordination Of Supports,2d005c1d-754f-11ea-ade9-022376bbb7c0,7d0af54b-f597-11e9-bfb4-022d4762bb3c,Support coordination,5874551b-f597-11e9-bfb4-022d4762bb3c,Capacity Building,ALL_PAID,PAID,100.14,25.0350,25.04,469336.82,32211.6876,27648.31
2631,2631,25703530-4f0a-11eb-aab6-0727412ff01c,2021-04,Shirline,Purvis,Brain Injury,3124802.0,ACT_NSW_QLD_VIC,99accb3d-0d8c-437b-9625-416d0e3d008e,PLAN_DELIVERY_ACTIVE,2020-12,2021-12,Consultation About Home Modification Designs W...,85e9dec0-754f-11ea-ade9-022376bbb7c0,7d0af64d-f597-11e9-bfb4-022d4762bb3c,Home,587454f6-f597-11e9-bfb4-022d4762bb3c,Capital,ALL_PAID,PAID,500.00,306.3000,306.30,68850.62,5995.9875,5996.08
5248,5248,5a853620-09f1-11eb-b7e6-5f90c3a4b38d,2021-04,Levy,Salle,"Depression, Acquired Brain Injury,",3131308.0,ACT_NSW_QLD_VIC,8ca47034-ef3c-45fa-8549-1aae25611155,PLAN_DELIVERY_ACTIVE,2020-10,2021-10,Coordination Of Supports,2d005c1d-754f-11ea-ade9-022376bbb7c0,7d0af54b-f597-11e9-bfb4-022d4762bb3c,Support coordination,5874551b-f597-11e9-bfb4-022d4762bb3c,Capacity Building,ALL_PAID,PAID,100.14,6208.4448,6208.44,4295076.49,172764.7611,174281.69
8843,8843,75cc8ab0-70b6-11eb-ad13-9558bc5f84e9,2021-04,Burch,Clelle,Intellectual disability,0.0,ACT_NSW_QLD_VIC,fc9cdde4-9da6-47f7-b9b2-0033dd1cebf5,PLAN_DELIVERY_ACTIVE,2020-08,2021-08,Coordination Of Supports,2d005c1d-754f-11ea-ade9-022376bbb7c0,7d0af54b-f597-11e9-bfb4-022d4762bb3c,Support coordination,5874551b-f597-11e9-bfb4-022d4762bb3c,Capacity Building,ALL_PAID,PAID,100.14,686.0000,686.00,558819.64,19550.5040,17708.21
9438,9438,7f73a110-e8f4-11ea-95c6-1ddffa748b78,2021-04,Clarita,Keizman,Polymyositits,0.0,ACT_NSW_QLD_VIC,b626259e-f3ce-4188-98e9-0d833f5b8825,PLAN_DELIVERY_ACTIVE,2021-01,2022-01,Assistive Products For Personal Care And Safety,ec637271-bb29-11ea-ade9-022376bbb7c0,7d0af628-f597-11e9-bfb4-022d4762bb3c,Assistive technology,587454f6-f597-11e9-bfb4-022d4762bb3c,Capital,ALL_PAID,PAID,5900.00,5900.0000,5900.00,140553.00,27075.1500,26722.36
10993,10993,94f0c500-cc92-11ea-9f69-1da30b9bef59,2021-04,Winslow,Bulfield,Motor neuron disease,3104121.0,ACT_NSW_QLD_VIC,19a7f7a6-a01b-47bd-8426-aca532b4ff31,PLAN_DELIVERY_ACTIVE,2020-12,2021-06,Coordination Of Supports,2d005c1d-754f-11ea-ade9-022376bbb7c0,7d0af54b-f597-11e9-bfb4-022d4762bb3c,Support coordination,5874551b-f597-11e9-bfb4-022d4762bb3c,Capacity Building,ALL_PAID,PAID,100.14,1516.1524,1950.76,147240.40,19160.7846,20631.01
17801,17801,bfcf9060-c7e9-11ea-80e0-3df355beb07c,2021-04,Concettina,Gridley,Intellectual disability,3124802.0,ACT_NSW_QLD_VIC,2d051ed1-a146-46b

In [ ]:
# Select just the fields to be used
main_dataset = main_dataset.groupby([
                                           "member_key"
                                         , "first_name"
                                         , "last_name"
                                         , "budget_level1_key"
                                         , "budget_level1_name"
                                         , "budget_level2_key"
                                         , "budget_level2_name"
                                         , "plan_start_date"
                                         , "plan_end_date"
                                         , "allocated_amount_total"
                                        ]).agg({"funded_amount": "sum", "allocated_amount": "sum"}).reset_index().copy()


In [ ]:
# Check the dataSet fields
print(f"Missing values in main_dataset each variable: \n{main_dataset.isnull().sum()}")
print(f"Duplicated rows: \n{main_dataset.duplicated().sum()}")
print(main_dataset.shape)

Missing values in main_dataset each variable: 
member_key                0
first_name                0
last_name                 0
budget_level1_key         0
budget_level1_name        0
budget_level2_key         0
budget_level2_name        0
plan_start_date           0
plan_end_date             0
allocated_amount_total    0
funded_amount             0
allocated_amount          0
dtype: int64
Duplicated rows: 
0
(12, 12)


In [ ]:
main_dataset

,member_key,first_name,last_name,budget_level1_key,budget_level1_name,budget_level2_key,budget_level2_name,plan_start_date,plan_end_date,allocated_amount_total,funded_amount,allocated_amount
0,04059890-e588-11ea-b453-3de7750d4ba6,Gretel,Moreno,5874551b-f597-11e9-bfb4-022d4762bb3c,Capacity Building,7d0af54b-f597-11e9-bfb4-022d4762bb3c,Support coordination,2020-08,2021-12,554391.70,2428.41,100.14
1,05b28540-e588-11ea-b453-3de7750d4ba6,Emilee,Styant,58745466-f597-11e9-bfb4-022d4762bb3c,Core,7d0af71b-f597-11e9-bfb4-022d4762bb3c,Assistance with daily living,2020-08,2021-08,99730.20,14315.00,1.00
2,13b3bd00-0485-11eb-aced-072c098d2335,Matthiew,Buckett,5874551b-f597-11e9-bfb4-022d4762bb3c,Capacity Building,7d0af54b-f597-11e9-bfb4-022d4762bb3c,Support coordination,2021-02,2022-02,469336.82,25.04,100.14
3,25703530-4f0a-11eb-aab6-0727412ff01c,Shirline,Purvis,587454f6-f597-11e9-bfb4-022d4762bb3c,Capital,7d0af64d-f597-11e9-bfb4-022d4762bb3c,Home,2020-12,2021-12,68850.62,306.30,500.00
4,5a853620-09f1-11eb-b7e6-5f90c3a4b38d,Levy,Salle,5874551b-f597-11e9-bfb4-022d4762bb3c,Capacity Building,7d0af54b-f597-11e9-bfb4-022d4762bb3c,Support coordination,2020-10,2021-10,4295076.49,6208.44,100.14
5,75cc8ab0-70b6-11eb-ad13-9558bc5f84e9,Burch,Clelle,5874551b-f597-11e9-bfb4-022d4762bb3c,Capacity Building,7d0af54b-f597-11e9-bfb4-022d4762bb3c,Support coordination,2020-08,2021-08,558819.64,686.00,100.14
6,7f73a110-e8f4-11ea-95c6-1ddffa748b78,Clarita,Keizman,587454f6-f597-11e9-bfb4-022d4762bb3c,Capital,7d0af628-f597-11e9-bfb4-022d4762bb3c,Assistive technology,2021-01,2022-01,140553.00,5900.00,5900.00
7,94f0c500-cc92-11ea-9f69-1da30b9bef59,Winslow,Bulfield,5874551b-f597-11e9-bfb4-022d4762bb3c,Capacity Building,7d0af54b-f597-11e9-bfb4-022d4762bb3c,Support coordination,2020-12,2021-06,147240.40,1950.76,100.14
8,bfcf9060-c7e9-11ea-80e0-3df355beb07c,Concettina,Gridley,5874551b-f597-11e9-bfb4-022d4762bb3c,Capacity Building,7d0af54b-f597-11e9-bfb4-022d4762bb3c,Support coordination,2021-01,2022-01,178148.70,375.39,100.14
9,c0f1c940-c7e9-11ea-80e0-3df355beb07c,Rory,Abrahmson,5874551b-f597-11e9-bfb4-022d4762bb3c,Capacity Building,7d0af54b-f597-11e9-bfb4-022d4762bb3c,Support coordination,2020-06,2021-06,412791.35,2941.80,98.06


In [ ]:
#Here we calculated total number of months in an active budegt plan
months_total = main_dataset[{
                              "member_key"
                            , "plan_start_date"
                            , "plan_end_date"}].copy()

months_total['plan_months_total'] = ((pd.to_datetime(months_total['plan_end_date']).dt.year
                                   -  pd.to_datetime(months_total['plan_start_date']).dt.year) * 12) + (pd.to_datetime(months_total['plan_end_date']).dt.month
                                   -  pd.to_datetime(months_total['plan_start_date']).dt.month)

months_total_merged = months_total[{"member_key", "plan_months_total"}]
                            
months_total_merged = months_total[{
                                    "member_key"
                                  , "plan_months_total"
                            }].groupby([
                                       "member_key"
                                     , "plan_months_total"
                                       ]).count().reset_index()

months_total_merged = pd.merge(main_dataset, months_total_merged, how="left", on=["member_key"])

In [ ]:
months_total_merged

,member_key,first_name,last_name,budget_level1_key,budget_level1_name,budget_level2_key,budget_level2_name,plan_start_date,plan_end_date,allocated_amount_total,funded_amount,allocated_amount,plan_months_total
0,04059890-e588-11ea-b453-3de7750d4ba6,Gretel,Moreno,5874551b-f597-11e9-bfb4-022d4762bb3c,Capacity Building,7d0af54b-f597-11e9-bfb4-022d4762bb3c,Support coordination,2020-08,2021-12,554391.70,2428.41,100.14,16
1,05b28540-e588-11ea-b453-3de7750d4ba6,Emilee,Styant,58745466-f597-11e9-bfb4-022d4762bb3c,Core,7d0af71b-f597-11e9-bfb4-022d4762bb3c,Assistance with daily living,2020-08,2021-08,99730.20,14315.00,1.00,12
2,13b3bd00-0485-11eb-aced-072c098d2335,Matthiew,Buckett,5874551b-f597-11e9-bfb4-022d4762bb3c,Capacity Building,7d0af54b-f597-11e9-bfb4-022d4762bb3c,Support coordination,2021-02,2022-02,469336.82,25.04,100.14,12
3,25703530-4f0a-11eb-aab6-0727412ff01c,Shirline,Purvis,587454f6-f597-11e9-bfb4-022d4762bb3c,Capital,7d0af64d-f597-11e9-bfb4-022d4762bb3c,Home,2020-12,2021-12,68850.62,306.30,500.00,12
4,5a853620-09f1-11eb-b7e6-5f90c3a4b38d,Levy,Salle,5874551b-f597-11e9-bfb4-022d4762bb3c,Capacity Building,7d0af54b-f597-11e9-bfb4-022d4762bb3c,Support coordination,2020-10,2021-10,4295076.49,6208.44,100.14,12
5,75cc8ab0-70b6-11eb-ad13-9558bc5f84e9,Burch,Clelle,5874551b-f597-11e9-bfb4-022d4762bb3c,Capacity Building,7d0af54b-f597-11e9-bfb4-022d4762bb3c,Support coordination,2020-08,2021-08,558819.64,686.00,100.14,12
6,7f73a110-e8f4-11ea-95c6-1ddffa748b78,Clarita,Keizman,587454f6-f597-11e9-bfb4-022d4762bb3c,Capital,7d0af628-f597-11e9-bfb4-022d4762bb3c,Assistive technology,2021-01,2022-01,140553.00,5900.00,5900.00,12
7,94f0c500-cc92-11ea-9f69-1da30b9bef59,Winslow,Bulfield,5874551b-f597-11e9-bfb4-022d4762bb3c,Capacity Building,7d0af54b-f597-11e9-bfb4-022d4762bb3c,Support coordination,2020-12,2021-06,147240.40,1950.76,100.14,6
8,bfcf9060-c7e9-11ea-80e0-3df355beb07c,Concettina,Gridley,5874551b-f597-11e9-bfb4-022d4762bb3c,Capacity Building,7d0af54b-f597-11e9-bfb4-022d4762bb3c,Support coordination,2021-01,2022-01,178148.70,375.39,100.14,12
9,c0f1c940-c7e9-11ea-80e0-3df355beb07c,Rory,Abrahmson,5874551b-f597-11e9-bfb4-022d4762bb3c,Capacity Building,7d0af54b-f597-11e9-bfb4-022d4762bb3c,Support coordination,2020-06,2021-06,412791.35,2941.80,98.06,12


In [ ]:
#Here we calculated how much a member supposed to spend by end of each month. So we divided total budget plan amount with total budget plan months.
allocated_amount_month = months_total_merged[{
                                              "member_key"
                                            , "budget_level1_key"
                                            , "budget_level2_key"
                                            , "plan_months_total"
                                            , "allocated_amount"
                                            }].copy()

allocated_amount_month["allocated_amount_month"] = allocated_amount_month["allocated_amount"] / allocated_amount_month["plan_months_total"]                                                    

allocated_amount_month = allocated_amount_month[{'member_key', 'budget_level1_key', 'budget_level2_key', 'allocated_amount_month'}]

months_total_merged_monthly = pd.merge(months_total_merged, allocated_amount_month, how="left", on=["member_key", "budget_level1_key", "budget_level2_key"])

In [ ]:
months_total_merged_monthly

,member_key,first_name,last_name,budget_level1_key,budget_level1_name,budget_level2_key,budget_level2_name,plan_start_date,plan_end_date,allocated_amount_total,funded_amount,allocated_amount,plan_months_total,allocated_amount_month
0,04059890-e588-11ea-b453-3de7750d4ba6,Gretel,Moreno,5874551b-f597-11e9-bfb4-022d4762bb3c,Capacity Building,7d0af54b-f597-11e9-bfb4-022d4762bb3c,Support coordination,2020-08,2021-12,554391.70,2428.41,100.14,16,6.258750
1,05b28540-e588-11ea-b453-3de7750d4ba6,Emilee,Styant,58745466-f597-11e9-bfb4-022d4762bb3c,Core,7d0af71b-f597-11e9-bfb4-022d4762bb3c,Assistance with daily living,2020-08,2021-08,99730.20,14315.00,1.00,12,0.083333
2,13b3bd00-0485-11eb-aced-072c098d2335,Matthiew,Buckett,5874551b-f597-11e9-bfb4-022d4762bb3c,Capacity Building,7d0af54b-f597-11e9-bfb4-022d4762bb3c,Support coordination,2021-02,2022-02,469336.82,25.04,100.14,12,8.345000
3,25703530-4f0a-11eb-aab6-0727412ff01c,Shirline,Purvis,587454f6-f597-11e9-bfb4-022d4762bb3c,Capital,7d0af64d-f597-11e9-bfb4-022d4762bb3c,Home,2020-12,2021-12,68850.62,306.30,500.00,12,41.666667
4,5a853620-09f1-11eb-b7e6-5f90c3a4b38d,Levy,Salle,5874551b-f597-11e9-bfb4-022d4762bb3c,Capacity Building,7d0af54b-f597-11e9-bfb4-022d4762bb3c,Support coordination,2020-10,2021-10,4295076.49,6208.44,100.14,12,8.345000
5,75cc8ab0-70b6-11eb-ad13-9558bc5f84e9,Burch,Clelle,5874551b-f597-11e9-bfb4-022d4762bb3c,Capacity Building,7d0af54b-f597-11e9-bfb4-022d4762bb3c,Support coordination,2020-08,2021-08,558819.64,686.00,100.14,12,8.345000
6,7f73a110-e8f4-11ea-95c6-1ddffa748b78,Clarita,Keizman,587454f6-f597-11e9-bfb4-022d4762bb3c,Capital,7d0af628-f597-11e9-bfb4-022d4762bb3c,Assistive technology,2021-01,2022-01,140553.00,5900.00,5900.00,12,491.666667
7,94f0c500-cc92-11ea-9f69-1da30b9bef59,Winslow,Bulfield,5874551b-f597-11e9-bfb4-022d4762bb3c,Capacity Building,7d0af54b-f597-11e9-bfb4-022d4762bb3c,Support coordination,2020-12,2021-06,147240.40,1950.76,100.14,6,16.690000
8,bfcf9060-c7e9-11ea-80e0-3df355beb07c,Concettina,Gridley,5874551b-f597-11e9-bfb4-022d4762bb3c,Capacity Building,7d0af54b-f597-11e9-bfb4-022d4762bb3c,Support coordination,2021-01,2022-01,178148.70,375.39,100.14,12,8.345000
9,c0f1c940-c7e9-11ea-80e0-3df355beb07c,Rory,Abrahmson,5874551b-f597-11e9-bfb4-022d4762bb3c,Capacity Building,7d0af54b-f597-11e9-bfb4-022d4762bb3c,Support coordination,2020-06,2021-06,412791.35,2941.80,98.06,12,8.171667


In [ ]:
#Here after calculating in which number of month the member is in of total budget plan, we calculate how much he/she should've already spent by now.
month_today = main_dataset[{
                           "member_key"
                         , "budget_level1_key"
                         , "budget_level2_key"
                         , "plan_start_date"
                            }].copy()


month_today['month_actual'] = ((pd.to_datetime("today").year -  pd.to_datetime(month_today['plan_start_date']).dt.year) * 12)                                                                                                                     + (pd.to_datetime("today").month - pd.to_datetime(month_today['plan_start_date']).dt.month)

month_today = month_today[{'member_key', 'budget_level1_key', 'budget_level2_key', 'month_actual'}]

month_actual = pd.merge(months_total_merged_monthly, month_today, how="left", on=["member_key", "budget_level1_key", "budget_level2_key"])

month_actual["spent_amount_predicted"] = month_actual["allocated_amount_month"] * month_actual["month_actual"]


In [ ]:
#rounded float values and calculated remaing months in a budget plan
month_actual['funded_amount'] = month_actual['funded_amount'].round(2)
month_actual['allocated_amount'] = month_actual['allocated_amount'].round(2)

month_actual['allocated_amount_month'] = month_actual['allocated_amount_month'].round(2)
month_actual['spent_amount_predicted'] = month_actual['spent_amount_predicted'].round(2)

month_actual['months_left'] = month_actual['plan_months_total'] - month_actual['month_actual']
month_actual

In [ ]:
#final_dataset = month_actual.drop(columns=['plan_key', 'status', 'level2_display_name', 'level1_name','plan_start_date', 'plan_start_date.1', 'id_budget', 'budget_number', 'item_category_level2_id', 'level2_key', 'level2_name', 'level1_id', 'level1_key','item_category_level3_key', 'level3_reference_number','name_stated_item', 'value_remained_budget', 'status_budget'])
# Rename
#


In [ ]:
month_actual['funded_amount_percentage'] = (month_actual['funded_amount'] / month_actual['allocated_amount']) 

month_actual['months_left_percentage'] = (month_actual['months_left'] / month_actual['plan_months_total']) 

month_actual['funded_amount_months_left'] = month_actual['funded_amount_percentage'] / month_actual['months_left_percentage']

month_actual = month_actual.loc[
                                (month_actual["plan_months_total"] >= 12)
                              & (month_actual["allocated_amount"] > 0)
                              & (month_actual["months_left"] > 0)
                               ]

In [ ]:
month_actual['spending_status'] = np.where(month_actual['funded_amount_months_left'] < 0.9, 'Overspending', 'Underspending')

month_actual['spending_status'] = np.where(((month_actual['funded_amount_months_left'] >= 0.9) & (month_actual['funded_amount_months_left'] <= 1.2)), 'On_Track', month_actual['spending_status'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
month_actual

,member_key,first_name,last_name,budget_level1_key,budget_level1_name,budget_level2_key,budget_level2_name,plan_start_date,plan_end_date,allocated_amount_total,funded_amount,allocated_amount,plan_months_total,allocated_amount_month,month_actual,spent_amount_predicted,months_left,funded_amount_percentage,months_left_percentage,funded_amount_months_left,spending_status
0,04059890-e588-11ea-b453-3de7750d4ba6,Gretel,Moreno,5874551b-f597-11e9-bfb4-022d4762bb3c,Capacity Building,7d0af54b-f597-11e9-bfb4-022d4762bb3c,Support coordination,2020-08,2021-12,554391.70,2428.41,100.14,16,6.26,10,62.59,6,24.250150,0.375000,64.667066,Underspending
1,05b28540-e588-11ea-b453-3de7750d4ba6,Emilee,Styant,58745466-f597-11e9-bfb4-022d4762bb3c,Core,7d0af71b-f597-11e9-bfb4-022d4762bb3c,Assistance with daily living,2020-08,2021-08,99730.20,14315.00,1.00,12,0.08,10,0.83,2,14315.000000,0.166667,85890.000000,Underspending
2,13b3bd00-0485-11eb-aced-072c098d2335,Matthiew,Buckett,5874551b-f597-11e9-bfb4-022d4762bb3c,Capacity Building,7d0af54b-f597-11e9-bfb4-022d4762bb3c,Support coordination,2021-02,2022-02,469336.82,25.04,100.14,12,8.35,4,33.38,8,0.250050,0.666667,0.375075,Overspending
3,25703530-4f0a-11eb-aab6-0727412ff01c,Shirline,Purvis,587454f6-f597-11e9-bfb4-022d4762bb3c,Capital,7d0af64d-f597-11e9-bfb4-022d4762bb3c,Home,2020-12,2021-12,68850.62,306.30,500.00,12,41.67,6,250.00,6,0.612600,0.500000,1.225200,Underspending
4,5a853620-09f1-11eb-b7e6-5f90c3a4b38d,Levy,Salle,5874551b-f597-11e9-bfb4-022d4762bb3c,Capacity Building,7d0af54b-f597-11e9-bfb4-022d4762bb3c,Support coordination,2020-10,2021-10,4295076.49,6208.44,100.14,12,8.35,8,66.76,4,61.997603,0.333333,185.992810,Underspending
5,75cc8ab0-70b6-11eb-ad13-9558bc5f84e9,Burch,Clelle,5874551b-f597-11e9-bfb4-022d4762bb3c,Capacity Building,7d0af54b-f597-11e9-bfb4-022d4762bb3c,Support coordination,2020-08,2021-08,558819.64,686.00,100.14,12,8.35,10,83.45,2,6.850409,0.166667,41.102457,Underspending
6,7f73a110-e8f4-11ea-95c6-1ddffa748b78,Clarita,Keizman,587454f6-f597-11e9-bfb4-022d4762bb3c,Capital,7d0af628-f597-11e9-bfb4-022d4762bb3c,Assistive technology,2021-01,2022-01,140553.00,5900.00,5900.00,12,491.67,5,2458.33,7,1.000000,0.583333,1.714286,Underspending
8,bfcf9060-c7e9-11ea-80e0-3df355beb07c,Concettina,Gridley,5874551b-f597-11e9-bfb4-022d4762bb3c,Capacity Building,7d0af54b-f597-11e9-bfb4-022d4762bb3c,Support coordination,2021-01,2022-01,178148.70,375.39,100.14,12,8.35,5,41.72,7,3.748652,0.583333,6.426260,Underspending
10,c26fce90-0090-11eb-8027-fd79b0c7803d,Netta,Mungan,587454f6-f597-11e9-bfb4-022d4762bb3c,Capital,7d0af628-f597-11e9-bfb4-022d4762bb3c,Assistive technology,2021-02,2022-02,263708.49,7541.96,7541.96,12,628.50,4,2513.99,8,1.000000,0.666667,1.500000,Underspending
11,c3213b80-d6d3-11ea-9cd1-f194648955a5,Fulton,Stott,5874551b-f597-11e9-bfb4-022d4762bb3c,Capacity Building,7d0af54b-f597-11e9-bfb4-022d4762bb3c,Support coordination,2020-12,2021-12,218190.79,175.25,100.14,12,8.35,6,50.07,6,1.750050,0.500000,3.500100,Underspending


In [ ]:
main_data_set = month_actual[{
                              "member_key"
                            , "first_name" 
                            , "last_name" 
                            , "budget_level1_key"
                            , "budget_level1_name"

# funded_amount
# plan_months_total

                            , "funded_amount_percentage"
                            , "months_left_percentage"
                            , "funded_amount_months_left"
                            
                            , "spending_status"
                            }].copy()

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
ord_enc = OrdinalEncoder()
main_data_set["spending_status"] = ord_enc.fit_transform(main_data_set[["spending_status"]])
main_data_set["spending_status"].nunique()

2

In [ ]:
df_model = main_data_set[{
                          'funded_amount_percentage'
                        , 'months_left_percentage'
                        , 'spending_status'
                        }].copy()

In [ ]:
df_model.head()

,spending_status,funded_amount_percentage,months_left_percentage
0,Underspending,24.250150,0.375000
1,Underspending,14315.000000,0.166667
2,Overspending,0.250050,0.666667
3,Underspending,0.612600,0.500000
4,Underspending,61.997603,0.333333


In [ ]:


X = df_model[{
                'funded_amount_percentage'
              , 'months_left_percentage'
            }]

y = df_model.spending_status


normalized_range = sklearn.preprocessing.MinMaxScaler(feature_range=(-1,1))
X = normalized_range.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.80, random_state=0)

In [ ]:
regression = LogisticRegression(random_state=0, multi_class='multinomial', penalty='none', solver='newton-cg', max_iter = 150, warm_start = False).fit(X_train, y_train)

regression.fit(X_train, y_train)

ValueError: ignored

In [ ]:
prediction_score = regression.score(X_test, y_test)
print('Prediction score: ', prediction_score * 100)

NameError: ignored

In [ ]:
print(regression.intercept_)

NameError: ignored

In [ ]:
print(regression.coef_)

NameError: ignored

In [ ]:
y_pred = regression.predict(X_test)

In [ ]:
y_pred = regression.predict(X_test)

confmtrx = np.array(confusion_matrix(y_test, y_pred))

pd.DataFrame(confmtrx, index=['Underspending','Overspending','On_Track'],
columns=['predicted_Underspending', 'predicted_Overspending', 'predicted_On_Track'])

In [ ]:
model_to_be_predicted = main_data_set[{
                                       'member_key'
                                     , 'first_name'
                                     , 'last_name'
                                     , 'budget_level1_key'
                                     , 'budget_level1_name'
                                     , 'funded_amount_percentage'
                                     , 'months_left_percentage'
                                     , 'spending_status'
                                    }].copy()

In [ ]:
df_prediction = pd.DataFrame(columns = ['member_key', 'first_name', 'last_name', 'budget_level1_key', 'budget_level1_name', 'prediction'])

for index in range(len(model_to_be_predicted)):
    member_key = (' member_key: ' + model_to_be_predicted['member_key'].iloc[index]
                + ' First name: ' + model_to_be_predicted['first_name'].iloc[index]
                + ' in Level 1:' + model_to_be_predicted['budget_level1_name'].iloc[index])

    funded_amount_percentage = model_to_be_predicted['funded_amount_percentage'].iloc[index]
    months_left_percentage = model_to_be_predicted['months_left_percentage'].iloc[index]
    spending_status = model_to_be_predicted['spending_status'].iloc[index]

    test = regression.predict((np.array([funded_amount_percentage, months_left_percentage]) .reshape(1, -1)))

    if test.astype(str) == 'Underspending':  
        status = 'Underspending'
    else:
        if test.astype(str) == 'Overspending':
            status = 'Overspending'
        else:
            status = 'On_Track'

    df = pd.DataFrame([ [
                         model_to_be_predicted['member_key'].iloc[index]
                       , model_to_be_predicted['first_name'].iloc[index]
                       , model_to_be_predicted['last_name'].iloc[index]
                       , model_to_be_predicted['budget_level1_key'].iloc[index]
                       , model_to_be_predicted['budget_level1_name'].iloc[index]
                       , status
                       ] ], columns=list(['member_key', 'first_name', 'last_name', 'budget_level1_key', 'budget_level1_name', 'prediction']))

    df_prediction = df_prediction.append(df)

df_prediction


In [ ]:
# split X and y into training and testing sets

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=0)

### Compute precision, recall, F-measure and support

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

NameError: ignored

### ROC Curve

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
logit_roc_auc = roc_auc_score(y_test, logreg.predict(x_test))
fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(x_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

In [ ]:
#compression_opts = dict(method='zip',
#                        archive_name='prediction.csv')

#df_prediction.to_csv('../dataSets/prediction.zip', compression=compression_opts)